In [1]:
function LUFactorization(dl, d, du)

    n = size(d, 1)

    ll = zeros(n - 1) # Lower diagonal of L
    l = zeros(n) # Main diagonal of L
    u = zeros(n - 1) # Upper diagonal of U
    p = ones(n) # Permutation matrix, A=PLU

    l[1] = d[1]
    u[1] = du[1] / l[1]

    for i in 2:(n - 1)

        ll[i - 1] = dl[i - 1]
        l[i] = d[i] - ll[i - 1] * u[i - 1]
        u[i] = du[i] / l[i]

    end

    ll[n - 1] = dl[n - 1]
    l[n] = d[n] - ll[n - 1] * u[n - 1]

    return ll, l, u

end

function Solver(ll, l, u, b)

    n = size(l, 1)

    z = zeros(n) # Solves Lz=b
    x = zeros(n) # The solution vector

    z[1] = b[1] / l[1]

    for i in 2:n

        z[i] = (b[i] - ll[i - 1] * z[i - 1]) / l[i]

    end

    x[n] = z[n]

    for i in reverse(1:(n - 1))

        x[i] = z[i] - u[i] * x[i + 1]

    end

    return x
end

Solver (generic function with 1 method)

In [2]:
using LinearAlgebra

n = 1000

# Generate random vectors to be used
dl = rand(n - 1)
d = rand(n)
du = rand(n - 1)
b = rand(n)

# Convert the tridiagonal to a full matrix
D = Tridiagonal(dl, d, du)

LU = LUFactorization(dl, d, du)

ll = LU[1]
l = LU[2]
u = LU[3]

L = Tridiagonal(ll, l, zeros(n - 1))
U = Tridiagonal(zeros(n - 1), ones(n), u)

x1 = Solver(ll, l, u, b)
x2 = LinearAlgebra.LAPACK.gtsv!(dl, d, du, b)

println(norm(x1 - x2, 2))

2.7547884159896566e-12


In [3]:
 using PyPlot

 maxn = 1000 # Larges size system to go to
n = maxn ÷ 10 # The size of the system
j = 10000 # The number of systems to solve at each size
dn = n # Change in n

# Defining the arrays 
arr0 = zeros(maxn ÷ dn)
arr1 = zeros(maxn ÷ dn)
arr2 = zeros(maxn ÷ dn)

while (n <= maxn)

    # Our implementation
    t1 = time()

    for i in 1:j

        dl = rand(n - 1)
        d = rand(n)
        du = rand(n - 1)
        b = rand(n)

        LU = LUFactorization(dl, d, du)

        ll = LU[1]
        l = LU[2]
        u = LU[3]

        x = Solver(ll, l, u, b)

    end

    arr1[n ÷ dn] = time() - t1

    # LAPACK.gtsv
    t2 = time()

    for i in 1:j

        dl = rand(n - 1)
        d = rand(n)
        du = rand(n - 1)
        b = rand(n)

        x = LinearAlgebra.LAPACK.gtsv!(dl, d, du, b)

    end

    arr1[n ÷ dn] = time() - t2

    arr0[n ÷ dn] = n
    n += dn
end
  
plot(arr0,arr1, label = "Our impementation")
plot(arr0,arr2, label = "LAPACK.gtsv")

PyPlot.xlabel("System Size")
PyPlot.ylabel("Time (s)")

PyPlot.title("LAPACK vs Our Implementation")
PyPlot.legend()
PyPlot.show()

sys:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
